In [1]:
import findspark
findspark.init()

from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

In [2]:
# Spark session & context
spark = SparkSession.builder.master("spark://spark:7077") \
        .appName("jupyter-notebook-index2mongo") \
        .config("spark.driver.memory", "512m") \
        .config("spark.mongodb.input.uri", "mongodb://mongodb:27017/test.myCollection") \
        .config("spark.mongodb.output.uri", "mongodb://mongodb:27017/test.myCollection") \
        .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.2') \
        .getOrCreate()

#         .config("spark.mongodb.write.connection.uri", "mongodb://mongodb:27017/test.myCollection") \
#         .config("spark.mongodb.read.connection.uri", "mongodb://mongodb:27017/test.myCollection") \
#         .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector:10.0.5') \

spark

# Write data in MongoDB

In [25]:
df = spark.read.parquet('/home/jovyan/work/data/**/*.parquet.zst')
df

DataFrame[language: string, country: string, job_id: string, productCategoryAlt: double, productAltEAN: double, productPriceBase: float, productBrand: string, productCategory: string, productDate: string, crawlID: string, currentURL: string, productLinkDetail: string, productEAN: string, productID: string, productAltId: string, productImage: string, productIsAvailable: bigint, productIsBio: bigint, productIsFresh: bigint, productIsFreezer: bigint, productIsNew: bigint, promotion: bigint, isSponsored: double, productLotPriceBase: float, productLotPrice: float, productName: string, productNutriscore: double, other: string, productPackaging: string, typeCrawler: string, productPosition: bigint, productPricePrevious: float, productPrice: float, productPromotionText: string, productQuantityRating: bigint, productRating: string, enseigneID: string, enseigneName: string, productService: double, shopID: string, productStock: bigint, productUnit: string, productValueUnit: float, variant_master:

In [ ]:
df.write.format("mongo").mode("append").save()
df.printSchema()

# Read data in MongoDB

In [7]:
df = spark.read.format("mongo").load()
df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- country: string (nullable = true)
 |-- crawlID: string (nullable = true)
 |-- currentURL: string (nullable = true)
 |-- enseigneID: string (nullable = true)
 |-- enseigneName: string (nullable = true)
 |-- job_id: string (nullable = true)
 |-- language: string (nullable = true)
 |-- other: string (nullable = true)
 |-- productBrand: string (nullable = true)
 |-- productCategory: string (nullable = true)
 |-- productDate: string (nullable = true)
 |-- productEAN: string (nullable = true)
 |-- productID: string (nullable = true)
 |-- productImage: string (nullable = true)
 |-- productIsAvailable: long (nullable = true)
 |-- productIsBio: long (nullable = true)
 |-- productIsFreezer: long (nullable = true)
 |-- productIsFresh: long (nullable = true)
 |-- productIsNew: long (nullable = true)
 |-- productLinkDetail: string (nullable = true)
 |-- productName: string (nullable = true)
 |-- productPackaging: s

In [13]:
df.select("productName").show()

+--------------------+
|         productName|
+--------------------+
|Petit-suisse 3.6% MG|
|Petit-suisse 9.2% MG|
|Yaourts aux fruit...|
|Petit-suisse 3.6% MG|
|Petit-suisse 9.2% MG|
|Lait demi-écrémé UHT|
|Cottage cheese fr...|
|Lait demi-écrémé UHT|
|CANDIA Grandlait ...|
|Yaourts aux fruit...|
|GILLETTE GILLETTE...|
|AOP Bordeaux Chât...|
|Sélection assorti...|
|       Yaourt nature|
|Yaourt nature bra...|
|Yaourt allégé aux...|
|   Skyr nature 0% MG|
|Yaourt allégé pan...|
|Lait demi-écrémé UHT|
|Lait demi-écrémé ...|
+--------------------+
only showing top 20 rows

